# Alunos
Thiago Pereira Freire - 202110167

Daniel Messias Santos - 202110168

# Enunciado

Suponha que temos uma tabela de nutrientes de diferentes tipos de alimentos. Sabendo o Valor Diário de Referência (VDR) de cada nutriente e o preço de cada unidade de alimento, qual é a dieta ótima que contém pelo menos o Valor Diário de Referência e possui o menor custo?

Para esta modelagem, considere $m_m$ nutrientes e nn alimentos. Defina $a_{ij}$ a quantidade do nutriente $i_i$ no alimento $j_j$, e $r_i$​ o valor diário mínimo de referência do nutriente $i_i$. Adicionalmente, seja $c_j$​ o preço do alimento $j_j$. Sua implementação do modelo de programação linear, utilizando o gurobipy, deve estabelecer as quantidades (em gramas) de cada alimento $j_j$ na dieta.

Como inputs, utilizaremos dados do McDonald's, já disponíveis no Campus Virtual. Os valores dos nutrientes são em % do VDR, por isso podemos usar simplesmente $r_i$ = 100%. As colunas que contêm um "NA" indicam que o valor "default" é usado. Logo, se "NA" aparece em um parâmetro que reflete um limite inferior, deve-se adotar o valor zero (0) como "default". Caso "NA" apareça em parâmetros de limite superior, então o valor "default" será um inteiro suficientemente grande (teoricamente: ∞).

$$ min f(c) = c_j $$

# Restrições

$$ \sum_{j=1}^n a_{ij} >= r_i $$

In [85]:
!pip install gurobipy

In [86]:
from gurobipy import Model, GRB
import pandas as pd

modelo = Model("dieta")

In [ ]:
# Ler o arquivo WSV usando pandas
nutrientes = pd.read_csv("McDonalds-amnt.wsv", sep='\s+', header=1, index_col=0)
custo = pd.read_csv("McDonalds-food.wsv", sep='\s+', header=2, index_col=0)
referencia = pd.read_csv("McDonalds-nutr.wsv", sep='\s+', header=1, index_col=0)

In [60]:
quantidade = []

for i in range(nutrientes.shape[0]):
  nome = nutrientes.index[i]  # Acessa o nome do alimento (primeira coluna)
  variavel = modelo.addVar(name=nome, vtype=GRB.CONTINUOUS, lb=0)
  quantidade.append(variavel)


## Função objetivo

In [61]:
modelo.setObjective(sum(quantidade[i] * int(custo.iloc[i, 0]) for i in range(len(quantidade))), GRB.MINIMIZE)

## Restrições

In [63]:
for j in range(nutrientes.shape[1]):
  modelo.addConstr(sum(quantidade[i] * int(nutrientes.iloc[i, j]) for i in range(len(quantidade))) >= referencia.iloc[j,0], f"minimo_{referencia.index[j]}")

In [64]:
modelo.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 20 rows, 254 columns and 1268 nonzeros
Model fingerprint: 0xb824f6f7
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 10 rows and 184 columns
Presolve time: 0.01s
Presolved: 10 rows, 70 columns, 588 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.500000e+01   0.000000e+00      0s
       4    2.0642202e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.064220183e+01


## Resultados

In [73]:
if(modelo.status == GRB.OPTIMAL):
  for i in range(len(quantidade)):
    if(quantidade[i].x > 0):
      print(quantidade[i].varName, quantidade[i].x)

McDuplo 5.045871559633028
Casquinha Baunilha 2.293577981651376
Maçã 3.2110091743119265


## Análise de sensibilidade

In [ ]:
print("\nIntervalos de Custos (Vetor de Custos):")
for variavel in modelo.getVars():
  if(variavel.SAObjLow != 0.0 and variavel.SAObjUp != "inf"):
    print(f"Variável {variavel.VarName}:")
    print(f"  lower bound do custo (SAObjLow): {variavel.SAObjLow}")
    print(f"  upper bound do custo (SAObjUp): {variavel.SAObjUp}")

print("\nIntervalos de Recursos (Vetor de Recursos):")
for restricao in modelo.getConstrs():
  print(f"Restrição {restricao.ConstrName}:")
  print(f"  lower bound do recurso (SARHSLow): {restricao.SARHSLow}")
  print(f"  upper bound do recurso (SARHSUp): {restricao.SARHSUp}")

### Explicação
A solução indica podemos comprar apenas MCDuplo, Casquinha de Baunilha e Maçã para atender os requisitos de nutrientes diários necessários.
Pela análise de sensibilidade, podemos observar qual o valor que cada um dos alimentos que não foram comprados devem assumir para alterar a solução, ou seja, qual o valor devem assumir para que valham a pena de serem comprados.

Além disso, pode-se notar que os nutrientes Colesterol e Carboidrato definem a quantidade de nutrientes a serem consumidos, pelo fato de que o limite inferior definido a eles é diferente de  -∞.


## Dieta vegetariana

In [87]:
modelo_veg = Model("dieta_vegetariana")

In [96]:
quantidade = []
food_index = []

for i in range(nutrientes.shape[0]):
  if(custo.iloc[i, 3] == True):
    food_index.append(i)
    nome = nutrientes.index[i]  # Acessa o nome do alimento (primeira coluna)
    variavel = modelo_veg.addVar(name=nome, vtype=GRB.CONTINUOUS, lb=0)
    quantidade.append(variavel)

In [98]:
modelo_veg.setObjective(sum(quantidade[i] * int(custo.iloc[food_index[i], 0]) for i in range(len(food_index))), GRB.MINIMIZE)

In [99]:
for j in range(nutrientes.shape[1]):
  modelo_veg.addConstr(sum(quantidade[i] * int(nutrientes.iloc[food_index[i], j]) for i in range(len(food_index))) >= referencia.iloc[j,0], f"minimo_{referencia.index[j]}")

In [107]:
modelo_veg.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 20 rows, 335 columns and 928 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.804878049e+01


In [108]:
if(modelo_veg.status == GRB.OPTIMAL):
  for i in range(len(quantidade)):
    if(quantidade[i].x > 0):
      print(quantidade[i].varName, quantidade[i].x)

Molho Salada Ranch 7.317073170731705
Casquinha Baunilha 13.658536585365857
Casquinha Chocolate 2.1951219512195115
Maçã 4.878048780487804


## Dieta com apenas uma unidade de cada alimento, ou nenhum

In [109]:
modelo_uni = Model("dieta_unitaria")

In [110]:
quantidade = []

for i in range(nutrientes.shape[0]):
  nome = nutrientes.index[i]  # Acessa o nome do alimento (primeira coluna)
  variavel = modelo_uni.addVar(name=nome, vtype=GRB.INTEGER, lb=0)
  quantidade.append(variavel)

In [111]:
modelo_uni.setObjective(sum(quantidade[i] * int(custo.iloc[i, 0]) for i in range(len(quantidade))), GRB.MINIMIZE)

In [112]:
for j in range(nutrientes.shape[1]):
  modelo_uni.addConstr(sum(quantidade[i] * int(nutrientes.iloc[i, j]) for i in range(len(quantidade))) >= referencia.iloc[j,0], f"minimo_{referencia.index[j]}")
for i in range(nutrientes.shape[0]):
  modelo_uni.addConstr(quantidade[i] <= 1, f"unidade_{i}")

In [113]:
modelo_uni.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 94 rows, 84 columns and 718 nonzeros
Model fingerprint: 0x4d6f14f3
Variable types: 0 continuous, 84 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 53.0000000
Presolve removed 85 rows and 5 columns
Presolve time: 0.01s
Presolved: 9 rows, 79 columns, 561 nonzeros
Variable types: 0 continuous, 79 integer (79 binary)

Root relaxation: objective 2.747302e+01, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.47

In [114]:
if(modelo_uni.status == GRB.OPTIMAL):
  for i in range(len(quantidade)):
    if(quantidade[i].x > 0):
      print(quantidade[i].varName, quantidade[i].x)

Quarterão com Queijo 1.0
McNífico Bacon 1.0
McDuplo 1.0
McBacon Junior 1.0
McFritas grande 1.0
McFritas pequena 1.0
Casquinha Baunilha 1.0
Casquinha Chocolate 1.0
Casquinha Mista 1.0
Maçã 1.0
